<h2>De connectie leggen tussen SSMS en Python</h2>

In [20]:
import pandas as pd
import pyodbc
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('..\\..\\sqllite\\go_sales.sqlite')
conn2 = sqlite3.connect('..\\..\\sqllite\\go_crm.sqlite')
conn3 = sqlite3.connect('..\\..\\sqllite\\go_staff.sqlite')

# Connect to SQL Server
DB = {'servername': 'LAPTOP-HSI9FLRD\\SQLEXPRESS',
      'database': 'DataWarehouseGreatOutdoors'}

export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] +
                             ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')

export_cursor = export_conn.cursor()

# properties
pd.set_option('display.max_columns', None)

<h2>Products table</h2>

In [4]:
# Read data from SQLite database
products = pd.read_sql_query("SELECT * FROM product", conn)
product_types = pd.read_sql_query("SELECT * FROM product_type", conn)
product_line = pd.read_sql_query("SELECT * FROM product_line", conn)

# Merge data into one table
combined_product_table = pd.merge(products, product_types, on='PRODUCT_TYPE_CODE')
combined_product_table = pd.merge(combined_product_table, product_line, on='PRODUCT_LINE_CODE')
combined_product_table

# Iterate through rows and insert into SQL Server
for index, row in combined_product_table.iterrows():
    try:
        query = "INSERT INTO Product (PRODUCT_NUMBER, PRODUCT_NAME, PRODUCT_DESCRIPTION, PRODUCT_PRODUCT_LINE_NAME, PRODUCT_PRODUCT_TYPE_NAME, PRODUCT_PRODUCT_COST_AMOUNT, PRODUCT_MARGIN_AMOUNT) VALUES (?, ?, ?, ?, ?, ?, ?)"
        export_cursor.execute(query, row['PRODUCT_NUMBER'], row['PRODUCT_NAME'], row['DESCRIPTION'], row['PRODUCT_LINE_EN'], row['PRODUCT_TYPE_EN'], row['PRODUCTION_COST'], row['MARGIN'])
    except pyodbc.Error as e:
        print(f"Error inserting row {index}: {e}")

export_conn.commit() 
# export_conn.close()

<h2>Satisfaction</h2>

In [14]:
# Read data from SQLite database
satisfaction = pd.read_sql_query("SELECT * FROM satisfaction", conn3)
satisfaction_type = pd.read_sql_query("SELECT * FROM satisfaction_type", conn3)

# Merge data into one table
combined_satisfaction_table = pd.merge(satisfaction, satisfaction_type, on='SATISFACTION_TYPE_CODE')

# Iterate through rows and insert into SQL Server
for index, row in combined_satisfaction_table.iterrows():
    try:
        query = "INSERT INTO Satisfaction (SATISFACTION_TYPE_CODE, SATISFACTION_TYPE_DESCRIPTION, SALES_STAFF_CODE, YEAR) VALUES (?, ?, ?, ?)"
        export_cursor.execute(query, row['SATISFACTION_TYPE_CODE'], row['SATISFACTION_TYPE_DESCRIPTION'], row['SALES_STAFF_CODE'], row['YEAR'])
    except pyodbc.Error as e:
        print(f"Error inserting row {index}: {e}")

export_conn.commit()

<h2>Training</h2>

In [22]:
# Read data from SQLite database
training = pd.read_sql_query("SELECT * FROM training", conn3)
course = pd.read_sql_query("SELECT * FROM course", conn3)

# Merge data into one table
combined_training_table = pd.merge(training, course, on='COURSE_CODE')

# Iterate through rows and insert into SQL Server
for index, row in combined_training_table.iterrows():
    try:
        query = "INSERT INTO Course (COURSE_CODE, COURSE_DESCRIPTION, SALES_STAFF_CODE, YEAR) VALUES (?, ?, ?, ?)"
        export_cursor.execute(query, row['COURSE_CODE'], row['COURSE_DESCRIPTION'], row['SALES_STAFF_CODE'], row['YEAR'])
    except pyodbc.Error as e:
        print(f"Error inserting row {index}: {e}")

export_conn.commit()  

<h2>Retailer table</h2>

In [92]:
# Read data from SQLite database
retailer = pd.read_sql_query("SELECT * FROM retailer", conn2)
retailer_type = pd.read_sql_query("SELECT * FROM retailer_type", conn2)
retailer_headquarters = pd.read_sql_query("SELECT * FROM retailer_headquarters", conn2)
retailer_site = pd.read_sql_query("SELECT * FROM retailer_site", conn)
country = pd.read_sql_query("SELECT * FROM country", conn2)

# Merge data into one table
combined_retailer_table = pd.merge(retailer, retailer_type, on='RETAILER_TYPE_CODE')
combined_retailer_table = pd.merge(combined_retailer_table, retailer_headquarters, on='RETAILER_CODEMR')
combined_retailer_table = pd.merge(combined_retailer_table, retailer_site, on= 'RETAILER_CODE')
combined_retailer_table = pd.merge(combined_retailer_table, country, left_on= 'COUNTRY_CODE_x', right_on= 'COUNTRY_CODE')
combined_retailer_table
# print(combined_retailer_table.columns)

# Iterate through rows and insert into SQL Server
for index, row in combined_retailer_table.iterrows():
    try:
        query = "INSERT INTO Retailer (RETAILER_CODE, RETAILER_CODE_MR, RETAILER_COMPANY_NAME, RETAILER_COUNTRY_NAME, RETAILER_POSTAL_ZONE_NAME, RETAILER_REGION_NAME, RETAILER_CITY_NAME, RETAILER_SALES_TERRITORY_NAME, RETAILER_ADDRESS_1_NAME, RETAILER_ADDRESS_2_NAME, RETAILER_RETAILER_TYPE, RETAILER_RETAILER_HEADQUARTERS, RETAILER_SITE_CODE) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
        export_cursor.execute(query, row['RETAILER_CODE'], row['RETAILER_CODEMR'], row['COMPANY_NAME'], 
                              row['COUNTRY_EN'], row['POSTAL_ZONE_x'], row['REGION_x'], row['CITY_x'], row['SALES_TERRITORY_CODE'], 
                              row['ADDRESS1_x'], row['ADDRESS2_x'], row['RETAILER_TYPE_CODE'], row['RETAILER_NAME'], row['RETAILER_SITE_CODE'])
    except pyodbc.Error as e:
        print(f"Error inserting row {index}: {e}") 

export_conn.commit()


<h2>Sales_staff table</h2>

In [93]:
# Read data from SQLite database
sales_staff = pd.read_sql_query("SELECT * FROM sales_staff", conn)
sales_branch = pd.read_sql_query("SELECT * FROM sales_branch", conn)
country = pd.read_sql_query("SELECT * FROM country", conn)

# Merge data into one table
combined_sales_staff_table = pd.merge(sales_staff, sales_branch, on='SALES_BRANCH_CODE')
combined_sales_staff_table = pd.merge(combined_sales_staff_table, country, on='COUNTRY_CODE')
combined_sales_staff_table

# Iterate through rows and insert into SQL Server
for index, row in combined_sales_staff_table.iterrows():
    try:
        # Adjust your INSERT query according to the actual column names in your SQL Server table
        query = "INSERT INTO SALES_STAFF ([Sales_staff_country_name], [Sales_staff_postal_zone_name], [Sales_staff_first_name], [Sales_staff_last_name], [Sales_staff_region_name], [Sales_staff_city_name], [Sales_staff_address_1_name], [Sales_staff_address_2_name], [Sales_staff_position_name], [Sales_staff_code]) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
        export_cursor.execute(query, row['COUNTRY'], row['POSTAL_ZONE'], row['FIRST_NAME'], row['LAST_NAME'], row['REGION'], row['CITY'], row['ADDRESS1'], row['ADDRESS2'], row['POSITION_EN'], row['SALES_STAFF_CODE'])
    except pyodbc.Error as e:
        print(f"Error inserting row {index}: {e}")

export_conn.commit()


<h2>Date table</h2>

<h2>order details table</h2>

In [94]:
# Read data from SQLite database
order_header = pd.read_sql_query("SELECT * FROM order_header", conn)
order_details = pd.read_sql_query("SELECT * FROM order_details", conn)
order_method = pd.read_sql_query("SELECT * FROM order_method", conn)


# Merge data into one table
combined_order_table = pd.merge(order_header, order_details, on='ORDER_NUMBER')
combined_order_table = pd.merge(combined_order_table, order_method, on='ORDER_METHOD_CODE')
combined_order_table

# Iterate through rows and insert into SQL Server
for index, row in combined_order_table.iterrows():
    try:
        # Adjust your INSERT query according to the actual column names in your SQL Server table
        query = "INSERT INTO ORDER_DETAILS ([order_detail_code], [order_number], [order_method_code], [order_method_en], [quantity], [order_detail_unit_cost], [order_detail_unit_price], [order_detail_unit_sale], [order_detail_order_date], [order_detail_product_number], [retailer_site_code], [sales_staff_code]) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"
        export_cursor.execute(query, row['ORDER_DETAIL_CODE'], row['ORDER_NUMBER'], row['ORDER_METHOD_CODE'], row['ORDER_METHOD_EN'], row['QUANTITY'], row['UNIT_COST'], row['UNIT_PRICE'], row['UNIT_SALE_PRICE'], row['ORDER_DATE'], row['PRODUCT_NUMBER'], row['RETAILER_SITE_CODE'], row['SALES_STAFF_CODE'])
    except pyodbc.Error as e:
        print(f"Error inserting row {index}: {e}")

export_conn.commit()


In [10]:
export_conn.close()